<h1>Deploying Hugging Face Models<h1>



In [1]:
!pip install transformers pandas torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
#upload car review file
from google.colab import files
uploaded = files.upload()

Saving car_reviews.csv to car_reviews.csv


In [3]:
import pandas as pd
df = pd.read_csv('car_reviews.csv')

<h2>NLP Model Selection for Review Classification</h2>

<p><strong>Model Chosen : </strong>facebook/bart-large-mnli</p>

<p><strong>Reasons : </strong></p>
  <ul>
    <li>This model is designed for Natural Language Inference (NLI) tasks and performs well in zero-shot classification.</li>
    <li>This model can classify texts into custom classes without retraining.</li>
    <li>This model can handle larger and complex texts.</li>
  </ul>

In [4]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
class_labels = [
    "Talks about driving experience",
    "Talks about features",
    "Talks about value for money",
    "Talks about issues",
    "Other"
]

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

def classify_reviews(reviews):
    results = classifier(reviews['Review'], candidate_labels=class_labels)
    return {
        "class": [result['labels'][0] for result in results],
        "score": [result['scores'][0] for result in results]
    }

classified_dataset = dataset.map(classify_reviews, batched=True, batch_size=128)

Map:   0%|          | 0/5959 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [7]:
print(classified_dataset)

Dataset({
    features: ['Review', 'class', 'score'],
    num_rows: 5959
})


In [8]:
print(classified_dataset['class'][:10])
print(classified_dataset['score'][:10])

['Talks about value for money', 'Talks about value for money', 'Talks about value for money', 'Talks about issues', 'Talks about value for money', 'Talks about issues', 'Talks about value for money', 'Talks about driving experience', 'Talks about driving experience', 'Talks about value for money']
[0.5374966859817505, 0.7576693892478943, 0.5380462408065796, 0.4684707522392273, 0.5198899507522583, 0.42335209250450134, 0.7301025390625, 0.381466805934906, 0.44012218713760376, 0.26754873991012573]


In [9]:
classified_df = classified_dataset.to_pandas()

assert len(df) == len(classified_df), "Row counts do not match!"

df['talks_about'] = classified_df['class']

In [10]:
df

,Review,talks_about
0,Bought 2017 Optima Hybrid in November 17. It w...,Talks about value for money
1,You get a lot for your money and great perfor...,Talks about value for money
2,This car is amazing and have no complaints. Y...,Talks about value for money
3,At 11k now in a lease for 39 months and it onl...,Talks about issues
4,"I've owned BMW, Lexus, Mercedes-Benz in the la...",Talks about value for money
...,...,...
5954,"Kia did a great job with this all new car, buy...",Talks about driving experience
5955,"When purchasing the car, I read that the newl...",Talks about value for money
5956,The new designed Kia Rio is awesome. Much room...,Talks about driving experience
5957,Everyone seems so hyped on having the latest &...,Talks about value for money
